In [1]:
import logging
import numpy as np  
from gensim.models import LdaModel
import os
import codecs

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',level=logging.INFO)

WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


In [2]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


In [3]:
from gensim import corpora, models, similarities

## Loading LDA

In [4]:
dictionary = corpora.Dictionary.load("/home/laampt/nlp/releases/docsent_695Kdoc.dict")

INFO:gensim.utils:loading Dictionary object from /home/laampt/nlp/releases/docsent_695Kdoc.dict


In [5]:
lda = LdaModel.load('/home/laampt/nlp/releases/docsent_695Kdoc_lda_bow_200topics_auto_50iterations_4workers_1passes_50iter_l1tok_r131')

INFO:gensim.utils:loading LdaModel object from /home/laampt/nlp/releases/docsent_695Kdoc_lda_bow_200topics_auto_50iterations_4workers_1passes_50iter_l1tok_r131
INFO:gensim.utils:loading id2word recursively from /home/laampt/nlp/releases/docsent_695Kdoc_lda_bow_200topics_auto_50iterations_4workers_1passes_50iter_l1tok_r131.id2word.* with mmap=None
INFO:gensim.utils:loading expElogbeta from /home/laampt/nlp/releases/docsent_695Kdoc_lda_bow_200topics_auto_50iterations_4workers_1passes_50iter_l1tok_r131.expElogbeta.npy with mmap=None
INFO:gensim.utils:setting ignored attribute state to None
INFO:gensim.utils:setting ignored attribute dispatcher to None
INFO:gensim.utils:loading LdaModel object from /home/laampt/nlp/releases/docsent_695Kdoc_lda_bow_200topics_auto_50iterations_4workers_1passes_50iter_l1tok_r131.state
INFO:gensim.utils:loading sstats from /home/laampt/nlp/releases/docsent_695Kdoc_lda_bow_200topics_auto_50iterations_4workers_1passes_50iter_l1tok_r131.state.sstats.npy with mmap

In [6]:
print dictionary
print lda

Dictionary(936365 unique tokens: [u'Kennycao', u'Shigaraki-ky\u014d', u'cmays', u'spiders', u'Nampo']...)
LdaModel(num_terms=936365, num_topics=200, decay=0.5, chunksize=2000)


## Load classifiers

In [7]:
from sklearn.externals import joblib

In [8]:
lr2 = joblib.load("/home/laampt/nlp/docsent/model/lda_lr_10Ktrain_cv10_200topics_r202.pkl")
svc2 = joblib.load("/home/laampt/nlp/docsent/model/lda_svc_10Ktrain_cv10_200topics_r202.pkl")
gbc2 = joblib.load("/home/laampt/nlp/docsent/model/lda_gbc_10Ktrain_cv10_200topics_r202.pkl")

## Sanity check

In [1]:
import re 
import string
import unicodedata
import sys
import codecs

STOPWORDS = [u"wwDATEww", u"wwTIMEww", u"wwEMAILww", u"wwIPww", u"wwURLww", u"wwNUMBERww"]
with codecs.open("/home/laampt/orm/10K/5K/lda/stopwords.txt", encoding='utf-8', mode='r') as fin:
        STOPWORDS += fin.readlines()
        
tbl = dict.fromkeys(i for i in xrange(sys.maxunicode)
                      if unicodedata.category(unichr(i)).startswith('P') and i != 45 and i!= 95)

def vi_trans_unicode(su):
    return su.translate(tbl)

vpunctuation = string.punctuation.replace('-','').replace('_','')
print vpunctuation

def vi_strip_text(s):
    s = re.sub(r"&amp;", "", s)
    s = re.sub(u"\u2026", "", s, re.UNICODE) # ...
    s = re.sub(u"\u2014", "", s, re.UNICODE) # emdash
    s = re.sub(u"\u201d", "", s, re.UNICODE) #
    s = re.sub(u"\u201c", "", s, re.UNICODE) #
    s = re.sub(r"<([^>]+)>", "", s)
    s = re.sub(r"(\s|\\n|\\r|\\t)+", " ", s)
    s = re.sub("([%s]+)" % vpunctuation, " ", s)
    s = ' '.join([w if not w[0].isdigit() else u"_NUM" for w in s.strip().split()])
    return s

def vi_strip_text2(s):
    s = re.sub(r"&amp;", "", s)
    s = vi_trans_unicode(s)
    s = re.sub(r"<([^>]+)>", "", s)
    s = re.sub(r"(\s|\\n|\\r|\\t)+", " ", s)
    s = re.sub(r"__+", "_", s)
    s = re.sub(r"--+", "-", s)
    s = ' '.join([w if not w[0].isdigit() else u"_NUM" for w in s.strip().split()])
    return s

def vi_clean(line):
    words = line.replace('.',' ').strip().split()
    words = [w for w in words if not(u"_NUM" in w) and not(w.startswith(u'-'))]
    return ' '.join(words)

def vi_clean2(line):
    words = line.replace('.',' ').strip().split()
    words = [w for w in words if not (w in STOPWORDS) and not(u"_NUM" in w) and not(w.startswith(u'-'))]
    return ' '.join(words)

import requests
from bs4 import BeautifulSoup
import json

url_post = "http://vlsp.vietlp.org:8080/demo/?page=seg_pos_chunk"
header_post = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/4",
    "Content-Type": "application/x-www-form-urlencoded; charset=UTF-8"
}

url_post_l1 = "http://192.168.0.125:8080/api/v1.0/document/filter?level=1"

def tokenize_soup(soup):
    try:
        sentences = soup.findAll('div', {'class': "sentence"})
        toks = [ s.find('tr').findAll('td')[1:] for s in sentences]
        toks = [t.contents[0] for tok in toks for t in tok]
        return ' '.join(toks)
    except Exception, e:
        print e
        
def get_tokens(INPUT):
    try:
        payload = {r"input":INPUT}
        rsp = requests.post(url_post, data=payload, headers=header_post)
        soup = BeautifulSoup(rsp.content)
        return tokenize_soup(soup)
    except Exception, e:
        print e
        
def get_tokens2(INPUT):
    try:
        rsp = requests.post(url_post_l1, data=INPUT)
        results = json.loads(rsp.content)
        return results["document"]
    except Exception, e:
        print e

!"#$%&'()*+,./:;<=>?@[\]^`{|}~


In [3]:
print get_tokens2("Nguỵ biện nói cho cùng là vấn đề chân lí và sự thật.")

Nguỵ_biện nói cho cùng là vấn_đề chân_lí và sự_thật.


In [10]:
def vectorize(doc, dictionary, lda):
    try:
        bow = dictionary.doc2bow(doc.split())
        tm = lda[bow]
        vec = tuple_to_vect(tm)
        return vec
    except Exception, e:
        print e
    
def voting_vec4(test_vec, cls):
    names = [type(clf).__name__ for clf in cls]
    scores = [clf.predict(test_vec) for clf in cls]
    return zip(names,scores)

In [12]:
def tuple_to_vect(tup, dim=200):
    vec = [0]*dim
    for k,v in tup:
        vec[k] = v
    return vec

In [15]:
INPUT = u"""Nguỵ biện nói cho cùng là vấn đề chân lí và sự thật. Vấn đề ở VN là chân lí nó được bóp méo và vặn vẹo làm cho người dân nghĩ rằng chỉ có một chân lí duy nhất. Quan sát trên báo chí, internet, đến nghị trường, từ những người có học (cỡ sư sĩ), đến chính khách, và cả thường dân ai ai cũng dùng những lí lẽ mà họ không biết là lỗi nguỵ biện. Chẳng hạn như gần đây nhất có người cảnh báo rằng coi chừng lợi dụng dân chủ, nhưng trong thực tế đó cũng là một dạng của nguỵ biện – nguỵ biện dựa vào nguồn tin vu vơ. Còn sự thật thì cũng bị giả tạo và giả dối sản sinh (như vụ Lê Văn Tám). Những thói nguỵ biện nó được xây dựng trên sự dối trá đã được tạo ra quá lâu, nên khó có thể chỉnh sửa một sớm một chiều. Chỉ khi nào nền giáo dục có sự tham gia bình đẳng từ các nhóm xã hội, thay vì bị sự kiểm soát của một thế lực chính trị, thì tình trạng nguỵ biện vẫn còn."""
test_vec = vectorize(INPUT, dictionary, lda)
voting_vec4(test_vec, [lr2,svc2,gbc2])

[('LogisticRegression', array([ 1.])),
 ('LinearSVC', array([ 1.])),
 ('GradientBoostingClassifier', array([ 1.]))]

In [20]:
INPUT = u"""Trong đoàn có lãnh_đạo các Bộ_An_ninh Quốc_gia Bộ_Nội_vụ cơ_quan_An_ninh Cảnh_sát các_nước Trung_Quốc Nga Cuba các_nước ASEAN lãnh_đạo tổ_chức Aseanpol và Interpol. Thay_mặt Chính_phủ Thủ_tướng_Nguyễn_Tấn_Dũng hoan_nghênh và đánh_giá_cao các đoàn_đại_biểu quốc_tế sang dự Lễ_kỷ_niệm wwNUMBERww năm Ngày truyền_thống CAND Việt_Nam và wwNUMBERww năm Ngày hội toàn_dân_bảo_vệ_an_ninh_Tổ_quốc. Sự có_mặt của các đại_biểu quốc_tế đã mang đến cho Việt_Nam nói_chung và cho lực_lượng CAND Việt_Nam nói_riêng sự gắn_bó tình_cảm hữu_nghị sự_hợp_tác tốt_đẹp. Thủ_tướng_Nguyễn_Tấn_Dũng bày_tỏ vui_mừng trước những bước phát_triển mạnh_mẽ trong quan_hệ_hợp_tác nhiều mặt giữa Việt_Nam với các_nước trong ASEAN Trung_Quốc Nga Cuba các tổ_chức_quốc_tế cũng như sự_hợp_tác hiệu_quả giữa Bộ_Công_an Việt_Nam với các lực_lượng_an_ninh cảnh_sát nội_vụ tình_báo của các_nước và_các tổ_chức Aseanpol Interpol."""
test_vec = vectorize(INPUT, dictionary, lda)
voting_vec4(test_vec, [lr2,svc2,gbc2])

[('LogisticRegression', array([ 0.])),
 ('LinearSVC', array([ 1.])),
 ('GradientBoostingClassifier', array([ 1.]))]

In [21]:
INPUT = u"""Tuần_trước ở Montreal tôi đã không_chơi tốt nhưng tôi luôn nỗ_lực. Tôi đã có_một kết_quả tích_cực trước Chardy. Vượt qua một đối_thủ như cậu_ấy là tin tốt_lành với tôi. Trận_đấu tiếp_theo sẽ còn rất khó_khăn."""
test_vec = vectorize(INPUT, dictionary, lda)
voting_vec4(test_vec, [lr2,svc2,gbc2])

[('LogisticRegression', array([ 0.])),
 ('LinearSVC', array([ 0.])),
 ('GradientBoostingClassifier', array([ 0.]))]

In [29]:
INPUT = u""""""
test_vec = vectorize(INPUT, dictionary, lda)
voting_vec4(test_vec, [lr2,svc2,gbc2])

[('LogisticRegression', array([ 1.])),
 ('LinearSVC', array([ 0.])),
 ('GradientBoostingClassifier', array([ 0.]))]

## Benchmark

In [23]:
%timeit lda[dictionary.doc2bow(u"Tuần_trước ở Montreal tôi đã không_chơi tốt nhưng tôi luôn nỗ_lực. Tôi đã có_một kết_quả tích_cực trước Chardy. Vượt qua một đối_thủ như cậu_ấy là tin tốt_lành với tôi. Trận_đấu tiếp_theo sẽ còn rất khó_khăn.".split())]

The slowest run took 4.69 times longer than the fastest. This could mean that an intermediate result is being cached 
1000 loops, best of 3: 648 µs per loop


In [25]:
%timeit lr2.predict(test_vec)

The slowest run took 4.08 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 3: 30.1 µs per loop


In [27]:
%timeit gbc2.predict(test_vec)

10000 loops, best of 3: 51.9 µs per loop


In [28]:
%timeit svc2.predict(test_vec)

The slowest run took 9.89 times longer than the fastest. This could mean that an intermediate result is being cached 
10000 loops, best of 3: 30.2 µs per loop
